<a href="https://colab.research.google.com/github/0xZee/llama-index-rag/blob/main/llama_index_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

INSTALL & LIB
---

In [ ]:
!pip install -q llama-index llama-index-llms-groq llama-index-embeddings-huggingface
!pip install -q llama-index-tools


In [2]:
from google.colab import userdata
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
#from llama_index.experimental.query_engine import PandasQueryEngine

from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

#from llama_index.llms.cohere import Cohere
#from llama_index.embeddings.cohere import CohereEmbedding

LLM / EMBED SETTING
---

In [ ]:
# Setting LLM
GROQ_API_KEY = userdata.get('GROQ_API_KEY')
model_llama = "llama3-8b-8192"
model_gemma = "gemma-7b-it"
model_mixtral = "mixtral-8x7b-32768"
model_whistper = "whisper-large-v3"

LLM_GROQ = Groq(model=model_llama, api_key=GROQ_API_KEY)
Settings.llm = LLM_GROQ

# Setting Embeddings
#EMBED_CO = CohereEmbedding(cohere_api_key=COHERE_API, model_name="embed-english-v3.0", input_type="search_query",)
EMBED_HF = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
Settings.embed_model = EMBED_HF

In [4]:
embeddings = EMBED_HF.get_text_embedding("Hello world")
print (len(embeddings))
print(embeddings[:5])

384
[0.015196100808680058, -0.022570667788386345, 0.0085471011698246, -0.07417059689760208, 0.0038364154752343893]


DATA and VECTORSTORE
---

In [5]:
# Data Source : WEF_Global_Risks_Report_2024.pdf (World Economic Forum Global risks reports 2024)
data = SimpleDirectoryReader(input_dir="./").load_data()

In [9]:
index = VectorStoreIndex.from_documents(data)

RAG
---

In [10]:
chat_engine = index.as_chat_engine(chat_mode="condense_question", verbose=True)

In [12]:
question = "what the doc is about ?"
response = chat_engine.chat(question)


Querying with: what the doc is about ?


In [13]:
print(response)

The document appears to be a report on global risks, covering various societal, technological, and economic threats and challenges facing the world.


In [15]:
response = chat_engine.chat("can you tell me more ?")

Querying with: Here is a rewritten standalone question that captures all relevant context from the conversation:

"What are some specific societal, technological, and economic threats and challenges discussed in the report, and can you provide more information about each of them?"


In [16]:
print(response)

The report highlights various societal, technological, and economic threats and challenges. Some specific examples include:

* Societal threats:
	+ Erosion of social cohesion
	+ Involuntary migration
	+ Failure of climate-change adaptation
	+ Cybercrime and cyber insecurity
	+ Censorship
* Technological threats:
	+ Adverse outcomes of artificial intelligence
	+ Existential threats to humanity from an "arms race" in experimental technologies
* Economic threats:
	+ Economic downturn
	+ Public debt
	+ Inflation
	+ Labour shortage
	+ Household debt
	+ Failure of climate-change adaptation
	+ Energy supply shortage
	+ Pollution (air, water, soil)
	+ Water-supply shortage
	+ State fragility
	+ Inequality (wealth, income)
	+ Chronic diseases and health conditions

These threats and challenges are interconnected and can have far-reaching consequences, making it essential to address them through global cooperation and collective action.


In [17]:
response = chat_engine.chat("can you give me some data in this report in a table format?")

Querying with: Here is a rewritten version of the follow-up message as a standalone question that captures all relevant context from the conversation:

"Can you provide a table summarizing the data on global risks and threats mentioned in the report, including societal, technological, and economic threats, and their interconnected consequences?"


In [18]:
print(response)

Here is the table summarizing the data on global risks and threats mentioned in the report:

| Country | 1st Risk | 2nd Risk | 3rd Risk | 4th Risk | 5th Risk |
| --- | --- | --- | --- | --- | --- |
| Spain | Economic downturn | Public debt | Erosion of social cohesion | Labour shortage | Inflation |
| Sri Lanka | Economic downturn | Labour shortage | Energy supply shortage | Inflation | Public debt |
| Sweden | Economic downturn | Energy supply shortage | Inflation | Involuntary migration | Failure of climate-change adaption |
| Switzerland | Labour shortage | Energy supply shortage | Economic downturn | Cybercrime and cyber insecurity | Erosion of social cohesion |
| Taiwan, China | Economic downturn | Energy supply shortage | Geoeconomic confrontation | Labour shortage | Inflation |
| Thailand | Economic downturn | Pollution (air, water, soil) | Labour shortage | Household debt | Inequality (wealth, income) |
| Tunisia | Economic downturn | Public debt | Water-supply shortage | State

In [20]:
response = chat_engine.chat("can you give me my last questions?")
print(response)

Querying with: Here is the rewritten standalone question that captures all relevant context from the conversation:

"Can you provide the table summarizing the data on global risks and threats mentioned in the report, similar to the one you provided earlier?"
Unfortunately, the provided context does not contain a table summarizing the data on global risks and threats. However, it does contain information about the survey sample composition, risk governance, and risk outlook.


In [21]:
# STREAMING

response = chat_engine.stream_chat("can you summarize the doc for me in 6 bullets ?")
for token in response.response_gen:
    print(token, end="")

Querying with: Here is a rewritten standalone question that captures all relevant context from the conversation:

"Can you summarize the report on global risks and threats in 6 bullets, highlighting the key societal, technological, and economic threats and challenges facing the world?"
Here are six bullets summarizing the report on global risks and threats:

• The world is facing a duo of dangerous crises: climate change and conflict, which are contributing to an unstable global order characterized by polarizing narratives, eroding trust, and insecurity.

• The report highlights the rise of global risks against the backdrop of "structural forces" such as geostrategic, demographic, and technological shifts, which are pushing the world to the limit and beyond.

• The report explores the most severe perceived risks to economies and societies over two and 10 years, including the possibility of a 3°C world, the peak of human development, and an explosion of criminality and corruption.

• Th

In [ ]:
# RESET CHAT SESSION
chat_engine.reset()